In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_path = '/Users/jk1/stroke_datasets/stroke_unit_dataset/per_value/Extraction_20220815'
admission_data_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_unit_dataset/data/stroke_registry/post_hoc_modified/stroke_registry_post_hoc_modified.xlsx'
patient_selection_path = '/Users/jk1/temp/opsum_extraction_output/high_frequency_data_patient_selection_with_details.csv'

In [ ]:
from preprocessing.variable_assembly.variable_database_assembly import assemble_variable_database

feature_df = assemble_variable_database(data_path, admission_data_path, patient_selection_path)

In [ ]:
from preprocessing.variable_assembly.relative_timestamps import transform_to_relative_timestamps

feature_df = transform_to_relative_timestamps(feature_df, drop_old_columns=False)

In [ ]:
feature_df[feature_df['relative_sample_date'] > 24*14]['case_admission_id'].unique()

In [ ]:
feature_df['relative_sample_date'].describe()

In [ ]:
feature_df['binned_sample_date'] = np.floor(feature_df['relative_sample_date'])

In [ ]:
feature_df.head(50)

In [ ]:
feature_df.groupby(['case_admission_id', 'binned_sample_date', 'sample_label']).count().reset_index()

In [ ]:
feature_df[feature_df['sample_label'] == 'oxygen_saturation'].groupby(['case_admission_id', 'binned_sample_date'])['value'].count().unstack()

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.pyplot import cm
import numpy as np


colors=cm.hsv(np.linspace(0,1,len(feature_df.sample_label.unique())))
for variable, color in zip(feature_df.sample_label.unique(), colors):

    variable_sampling_count_df = feature_df[feature_df['sample_label'] == variable].groupby(['case_admission_id', 'binned_sample_date']).count().reset_index()

    fig = plt.figure(figsize=(10,4))
    ax1 = fig.add_subplot(121)
    sns.scatterplot(x='binned_sample_date' , y='value', data=variable_sampling_count_df,
                    color=color, alpha=0.02, ax=ax1)
    ax1.set_xlim(0, 24*14)
    ax1.set_title(f'{variable} sampling rate')
    ax1.set_ylabel('Sample counts')
    ax1.set_xlabel('Hourly relative sampling times')

    dfxx = variable_sampling_count_df.value.describe().to_frame().round(2)

    ax2 = fig.add_subplot(122)
    font_size=12
    bbox=[0, 0, 1, 1]
    ax2.axis('off')
    mpl_table = ax2.table(cellText = dfxx.values, rowLabels = dfxx.index, bbox=bbox, colLabels=['Sampling distribution'])
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)
    fig.savefig(f'{variable.replace("/","")}.png')


plt.show()

In [ ]:
variable_sampling_count_df = feature_df.groupby(['binned_sample_date']).count().reset_index()

fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(121)
sns.scatterplot(x='binned_sample_date' , y='value', data=variable_sampling_count_df,
                color=color, alpha=0.02, ax=ax1)
ax1.set_xlim(0, 24*14)
ax1.set_title(f'Variables sampled per hour')
ax1.set_ylabel('Sample counts')
ax1.set_xlabel('Hours from admission')

dfxx = variable_sampling_count_df.value.describe().to_frame().round(2)

ax2 = fig.add_subplot(122)
font_size=12
bbox=[0, 0, 1, 1]
ax2.axis('off')
mpl_table = ax2.table(cellText = dfxx.values, rowLabels = dfxx.index, bbox=bbox, colLabels=['Sampling distribution'])
mpl_table.auto_set_font_size(False)
mpl_table.set_fontsize(font_size)
fig.savefig(f'overall.png', dpi=700)

In [ ]:
feature_df[feature_df['sample_label'] == 'oxygen_saturation'].groupby(['case_admission_id', 'binned_sample_date']).count().reset_index()['value'].describe()

Check highly sampled individual

In [ ]:
temp = feature_df[feature_df['sample_label'] == 'oxygen_saturation'].groupby(['case_admission_id', 'binned_sample_date']).count().reset_index()
temp[temp['value'] == 121]

In [ ]:
feature_df[(feature_df['case_admission_id'] == '263647_2692')
           & (feature_df['sample_label'] == 'oxygen_saturation')
           & (feature_df['binned_sample_date'] == 455)]